In [5]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import scipy.stats as st
from pprint import pprint
import time
from scipy.stats import linregress
import reverse_geocoder as rg

In [2]:

# Set base url parameters
base_url = "https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson"
magnitude = "minmagnitude=5"
limit = "limit=19999"
year=10

# set up lists to hold reponse info
time_ls = []
lat_ls = []
lng_ls = []
depth_ls = []
mag_ls = []
mmi_ls = []
sig_ls = []
felt_ls = []
place_ls = []

# iterate through years
for x in range (0,11):
    time = f"starttime=20{year}-12-01&endtime=20{year}-12-02"
    query_url = (f"{base_url}&{time}&{magnitude}&{limit}")
    response = requests.get(query_url).json()

    # Append data to appropriate list
    for i in range (0, len(response["features"])):
        mag_ls.append(response["features"][i]["properties"]["mag"])
        place_ls.append(response["features"][i]["properties"]["place"])
        time_ls.append(response["features"][i]["properties"]["time"])
        felt_ls.append(response["features"][i]["properties"]["felt"])
        mmi_ls.append(response["features"][i]["properties"]["mmi"])
        sig_ls.append(response["features"][i]["properties"]["sig"])
        lat_ls.append(response["features"][i]["geometry"]["coordinates"][1])
        lng_ls.append(response["features"][i]["geometry"]["coordinates"][0])
        depth_ls.append(response["features"][i]["geometry"]["coordinates"][2])
    # counter for next year
    year=year+1
    

In [6]:
# create dictionary
data = {
    "Time": time_ls,
    "Lat": lat_ls,
    "Lng": lng_ls,
    "Depth": depth_ls,
    "Mag": mag_ls,
    "MMI": mmi_ls,
    "Sig": sig_ls,
    "Felt": felt_ls,
    "Place": place_ls
}
# create dataframe
data_df = pd.DataFrame(data)

#convert timestamp to dates
data_df["Time"] = pd.to_datetime(data_df["Time"], unit="ms")
data_df["Month"] = pd.DatetimeIndex(data_df["Time"]).month
data_df["Day"] = pd.DatetimeIndex(data_df["Time"]).day
data_df["Year"] = pd.DatetimeIndex(data_df["Time"]).year

#add country
data_df["Country"] = ""

for index, row in data_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    coordinates = (lat,lng)
    result = rg.search(coordinates)
    data_df.loc[index,"Country"] = result[0]["cc"]

Loading formatted geocoded file...


In [7]:
# Print dataframe

data_df= data_df[["Lat","Lng","Depth","Mag","MMI","Sig","Felt","Place","Country","Time","Month","Day","Year"]]

data_df.head()

,Lat,Lng,Depth,Mag,MMI,Sig,Felt,Place,Country,Time,Month,Day,Year
0,-15.888,-178.956,15.2,6.1,NaN,572,NaN,Fiji region,WF,2010-12-01 16:01:27.440,12,1,2010
1,2.689,99.041,160.1,5.6,3.663,482,NaN,"northern Sumatra, Indonesia",ID,2010-12-01 00:50:21.730,12,1,2010
2,-22.717,-174.858,57.4,5.1,NaN,400,NaN,Tonga region,TO,2010-12-01 00:09:45.330,12,1,2010
3,-0.626,121.974,10.0,5.5,5.611,466,4.0,"Sulawesi, Indonesia",ID,2011-12-01 22:53:36.500,12,1,2011
4,32.175,-40.295,10.0,5.4,NaN,449,NaN,northern Mid-Atlantic Ridge,PT,2011-12-01 21:35:09.740,12,1,2011


In [8]:
data_df.to_csv("earthquake_data_2010-20.csv", index= False)